In [1]:
# main
import pygame
import pygame.gfxdraw # для расширенных графических операций (сглаживание)
import json
import math

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:
pygame.init() # Инициализирует все модули Pygame
screen = pygame.display.set_mode((config['window_width'], config['window_height'])) # Создает главное окно приложения
pygame.display.set_caption("Гравитационное линзирование черной дыры") # Устанавливает заголовок окна
clock = pygame.time.Clock() # Создает объект для контроля частоты кадров (FPS)

In [4]:
smooth_surface = pygame.Surface((config['window_width'], config['window_height']), pygame.SRCALPHA) #smooth_surface - дополнительная поверхность с альфа-каналом для сглаживания графики и устранения визуальных артефактов

In [5]:
class BlackHole:
    def __init__(self):
        self.x = config['window_width'] // 2  # Устанавливает X-координату центра черной дыры по горизонтали
        self.y = config['window_height'] // 2 - 30  # Устанавливает Y-координату центра черной дыры (смещён вверх на 30)
        self.mass = config['initial_mass']  # Начальная масса
        self.base_radius = config['core_radius']  # Базовый радиус ядра
        self.radius = self.base_radius  # Текущий радиус Это радиус, который будет меняться при изменении массы
        self.gap_between_core_and_ring = 10  # расстояние между ядром и желтым кольцом

    def update_radius(self):
        # радиус Шварцшильда пропорционален массе : R = 2GM/c²
        # Отношение текущей массы к начальной
        mass_ratio = self.mass / config['initial_mass']
        # Радиус изменяется пропорционально массе
        self.radius = int(self.base_radius * mass_ratio)
    def draw_rings(self, surface):
        # Серое кольцо — внешнее, с учётом зазора
        pygame.draw.circle(surface, (100, 100, 100), (self.x, self.y),
                           self.radius + self.gap_between_core_and_ring +
                           config['yellow_ring_width'] * 2 + config['gray_ring_width'],
                           config['gray_ring_width'])
        # Желтое кольцо — на расстоянии от ядра
        pygame.draw.circle(surface, (255, 255, 0), (self.x, self.y),
                           self.radius + self.gap_between_core_and_ring + config['yellow_ring_width'],
                           config['yellow_ring_width'])
    
    def draw_lines(self, surface):
        # Центральная линия
        pygame.draw.line(surface, (150, 150, 150),
                         (0, self.y),
                         (config['window_width'], self.y), 2)
        # Верхняя линия
        line_y = self.y - (self.radius + self.gap_between_core_and_ring + config['yellow_ring_width'] * 1.5)
        pygame.draw.line(surface, (150, 150, 150),
                         (0, line_y),
                         (config['window_width'], line_y), 2)
    
    def draw_core(self, surface):
        # Чёрное ядро
        pygame.draw.circle(surface, (0, 0, 0), (self.x, self.y), self.radius)


In [6]:
class Photon:
    def __init__(self, start_x, start_y):
        self.start_x = start_x
        self.start_y = start_y
        self.reset() # инициализирует все остальные параметры фотона
        
    def reset(self): # Сбрасывает фотон в начальное состояние - возвращает к исходным параметрам.
        self.x = self.start_x
        self.y = self.start_y
        self.trail = [] # Очистка трейла (следа)
        self.trail_length = 80
        self.speed_x = -config['photon_speed']
        self.speed_y = 0
        self.active = True # Пока active = True, фотон будет обновляться и отрисовываться
        
        # Заполняем трейл до старта чтобы Плавный старт - при начале симуляции сразу видна траектория
        for i in range(self.trail_length):
            trail_x = self.x + (self.trail_length - i) * config['photon_speed']
            self.trail.append((trail_x, self.y))
    
    def update(self, black_hole): # Обновляет состояние фотона на каждом кадре - движение, гравитация, траектория.
        if not self.active:
            return
        
        # Добавляем позицию в трейл
        self.trail.append((self.x, self.y)) # Добавление текущей позиции  в трейл 
        if len(self.trail) > self.trail_length: # Если точек больше 80, удаляется САМАЯ СТАРАЯ точка
            self.trail.pop(0)

        dx = black_hole.x - self.x   # разность по X между черной дырой и фотоном
        dy = black_hole.y - self.y
        distance = math.sqrt(dx*dx + dy*dy) # Расстояние = √(Δx² + Δy²) - теорема Пифагора

        if distance > 0:
            # Ньютоновская гравитация: F = G * M / r^2
            force = (black_hole.mass * config['gravity_strength']) / (distance * distance)

            # Нормализованный вектор к центру - Вектор длины 1, показывающий НАПРАВЛЕНИЕ к черной дыре
            nx = dx / distance # X-компонента направления (от -1 до 1)
            ny = dy / distance

            # Обновляем скорость фотона - Ускорение = Сила × Направление
            self.speed_x += force * nx
            self.speed_y += force * ny
        
            # Ограничиваем максимальную скорость
            speed = math.sqrt(self.speed_x ** 2 + self.speed_y ** 2)
            if speed > config['max_photon_speed']:
                self.speed_x = self.speed_x / speed * config['max_photon_speed']
                self.speed_y = self.speed_y / speed * config['max_photon_speed']
        
        # Обновляем позицию
        self.x += self.speed_x
        self.y += self.speed_y

    def draw(self, surface):
        if not self.active:
            return
            
        if len(self.trail) > 1:
            pygame.draw.lines(surface, (255, 0, 0), False, self.trail, 1)
        if len(self.trail) > 0:
            pygame.draw.circle(surface, (255, 0, 0),
                               (int(self.trail[-1][0]), int(self.trail[-1][1])), 2)


In [7]:
class Slider:
    def __init__(self, x, y, width, height, min_val, max_val, initial):
        self.rect = pygame.Rect(x, y, width, height) #прямоугольная область слайдера с координатами (x,y) и размерами (width,height)
        self.min_val = min_val
        self.max_val = max_val
        self.value = initial #текущее значение массы (начинается с initial)
        self.dragging = False #флаг, отслеживающий перетаскивается ли слайдер
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos): #При нажатии кнопки мыши проверяет, попадает ли курсор в область слайдера.
            self.dragging = True # Если да - начинает перетаскивание.
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False #При отпускании кнопки мыши прекращает перетаскивание.
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            relative_x = event.pos[0] - self.rect.x
            self.value = self.min_val + (relative_x / self.rect.width) * (self.max_val - self.min_val)
            self.value = max(self.min_val, min(self.max_val, self.value))
            return True
        return False
    
    def draw(self, surface): #отрисовка слайдера
        pygame.draw.rect(surface, (50, 50, 50), self.rect)
        fill_width = int((self.value - self.min_val) / (self.max_val - self.min_val) * self.rect.width) #вычисляет ширину заполненной области пропорционально текущему значению
        pygame.draw.rect(surface, (100, 100, 255), (self.rect.x, self.rect.y, fill_width, self.rect.height)) #Отрисовка заполненной области

        slider_x = self.rect.x + fill_width
        pygame.draw.circle(surface, (200, 200, 255), (slider_x, self.rect.centery), 8) #Рисует светло-синий ползунок в позиции, соответствующей текущему значению.
        
        font = pygame.font.SysFont(None, 24) #Создает и отображает черный текст с текущим значением массы над слайдером.
        text = font.render(f"Масса черной дыры: {self.value:.0f}", True, (0, 0, 0))
        surface.blit(text, (self.rect.x, self.rect.y - 25))

        fps = clock.get_fps()
        fps_text = font.render(f"FPS: {fps:.1f}", True, (0, 0, 0))
        screen.blit(fps_text, (config['window_width'] - 100, 10))


In [8]:
#Создание черной дыры и слайдера
black_hole = BlackHole()
slider = Slider(config['slider_x'], config['slider_y'],
                config['slider_width'], config['slider_height'],
                config['min_mass'], config['max_mass'], config['initial_mass'])

photons = []
center_y = black_hole.y

num_photons = 25       
spacing = 6            # расстояние между фотонами 

#Создание параллельного пучка фотонов
for i in range(num_photons):
    y = center_y - (i * spacing) # размещение фотонов ВЫШЕ центра
    start_x = config['window_width']
    photons.append(Photon(start_x, y))

In [9]:
running = True
restart_interval = 5000  # каждые 5 секунд
last_restart_time = pygame.time.get_ticks()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: #проверяет событие закрытия окна (крестик)
            running = False
        slider.handle_event(event)
    
    # Автоматический перезапуск фотонов каждые 10 секунд
    current_time = pygame.time.get_ticks()
    if current_time - last_restart_time > restart_interval: #ычисляет сколько прошло времени с последнего перезапуска
        for photon in photons:
            photon.reset()
        last_restart_time = current_time

    # Обновление массы и радиуса чёрной дыры
    black_hole.mass = slider.value #обновляет массу черной дыры из значения слайдера
    black_hole.update_radius()

    # Обновление фотонов
    for photon in photons:
        photon.update(black_hole)

    smooth_surface.fill((0, 0, 0, 0))
    black_hole.draw_rings(smooth_surface)
    for photon in photons:
        photon.draw(smooth_surface)
    black_hole.draw_lines(smooth_surface)
    black_hole.draw_core(smooth_surface)
    # Отображение на экране
    screen.fill(config['background_color'])
    screen.blit(smooth_surface, (0, 0))
    slider.draw(screen)

    pygame.display.flip()
    clock.tick(config['animation_speed']) #ограничение FPS (60 кадров/сек)

pygame.quit()

: 